In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import random
import json
import sys

# -------------------- Utility Functions --------------------
def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculates MAPE, preventing division by zero.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    epsilon = 1e-10 # Small epsilon to prevent division by zero
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# -------------------- Device Setup --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------- Data Loading and Preprocessing --------------------
# Reads the 'review_data_optimized.parquet' file.
try:
    df = pd.read_parquet('review_data_optimized.parquet', engine='pyarrow')
    df_processed = df[['user_id', 'business_id', 'stars']].copy()

    # Converts user_id and business_id to integers using Label Encoding.
    user_encoder = LabelEncoder()
    business_encoder = LabelEncoder()
    df_processed['user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
    df_processed['business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

    num_users = len(user_encoder.classes_)
    num_businesses = len(business_encoder.classes_)

    print("✅ Data loaded and preprocessed successfully.")

except FileNotFoundError:
    print("❌ Error: 'review_data_optimized.parquet' not found.")
    sys.exit()

# -------------------- Dataset Definition --------------------
class NeuMFDataset(Dataset):
    def __init__(self, user_ids, item_ids, ratings):
        self.user_ids = torch.tensor(user_ids, dtype=torch.long)
        self.item_ids = torch.tensor(item_ids, dtype=torch.long)
        self.ratings = torch.tensor(ratings, dtype=torch.float)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

# -------------------- Model Definition --------------------
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, mf_dim=16, mlp_dims=[64, 32]):
        super(NeuMF, self).__init__()
        self.user_embedding_gmf = nn.Embedding(num_users, mf_dim)
        self.item_embedding_gmf = nn.Embedding(num_items, mf_dim)
        
        self.user_embedding_mlp = nn.Embedding(num_users, mlp_dims[0] // 2)
        self.item_embedding_mlp = nn.Embedding(num_items, mlp_dims[0] // 2)

        mlp_layers = []
        input_dim = mlp_dims[0]
        for dim in mlp_dims[1:]:
            mlp_layers.append(nn.Linear(input_dim, dim))
            mlp_layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*mlp_layers)

        self.final_layer = nn.Linear(mf_dim + mlp_dims[-1], 1)

    def forward(self, user_ids, item_ids):
        gmf_user = self.user_embedding_gmf(user_ids)
        gmf_item = self.item_embedding_gmf(item_ids)
        gmf_output = gmf_user * gmf_item

        mlp_user = self.user_embedding_mlp(user_ids)
        mlp_item = self.item_embedding_mlp(item_ids)
        mlp_input = torch.cat((mlp_user, mlp_item), dim=1)
        mlp_output = self.mlp(mlp_input)

        concat = torch.cat((gmf_output, mlp_output), dim=1)
        prediction = self.final_layer(concat)
        return prediction.view(-1)

# -------------------- Evaluation Function --------------------
def evaluate_model(model, data_loader, device):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for user_ids, item_ids, ratings in data_loader:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            output = model(user_ids, item_ids)
            preds.extend(output.cpu().numpy())
            targets.extend(ratings.cpu().numpy())
    
    preds = np.array(preds)
    targets = np.array(targets)

    mae = mean_absolute_error(targets, preds)
    mse = mean_squared_error(targets, preds)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(targets, preds)

    return mae, mse, rmse, mape

# -------------------- Hyperparameter Search Setup --------------------
param_grid = {
    'mf_dim': [8, 16, 32],
    'mlp_dims': [[64, 32], [128, 64], [128, 64, 32]],
    'learning_rate': [0.0005, 0.001, 0.002],
    'batch_size': [128, 256, 512],
    'patience': [5, 7, 10]
}

num_trials = 10
best_params = None
best_rmse = float('inf')
results_log = []
min_delta = 0.0001
epochs = 50
DATA_SPLIT_RANDOM_STATE = 42

print(f"\n--- Starting Randomized Hyperparameter Search with {num_trials} trials ---")
print(f"Data Split Random State: {DATA_SPLIT_RANDOM_STATE}")

# Perform data splitting ONCE to ensure fair comparison
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=DATA_SPLIT_RANDOM_STATE)
val_size_ratio = 1/8
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=DATA_SPLIT_RANDOM_STATE)

print(f"Fixed Data Split: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")

# Create DataLoader for test set once
test_dataset = NeuMFDataset(test_df['user_encoded'].values, test_df['business_encoded'].values, test_df['stars'].values)
# Use a reasonable batch size for the test loader, or match the one from the best trial
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False) 

# --- Randomized Search Loop ---
for trial_num in range(num_trials):
    print(f"\n==================== Trial {trial_num + 1}/{num_trials} ====================")
    
    # Randomly select hyperparameters for the current trial
    current_params = {k: random.choice(v) for k, v in param_grid.items()}
    print(f"Current Parameters: {current_params}")
    
    # Create DataLoaders for the current trial's batch size
    train_dataset = NeuMFDataset(train_df['user_encoded'].values, train_df['business_encoded'].values, train_df['stars'].values)
    val_dataset = NeuMFDataset(val_df['user_encoded'].values, val_df['business_encoded'].values, val_df['stars'].values)

    train_loader = DataLoader(train_dataset, batch_size=current_params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=current_params['batch_size'], shuffle=False)

    # Initialize model with current trial's parameters
    model = NeuMF(num_users, num_businesses, mf_dim=current_params['mf_dim'], mlp_dims=current_params['mlp_dims']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=current_params['learning_rate'])
    criterion = nn.MSELoss()
    
    trial_model_save_path = f'temp_neumf_model_trial_{trial_num+1}.pt'
    trial_best_val_rmse = float('inf')
    epochs_no_improve = 0
    patience = current_params['patience']

    # --- Training Loop for current trial ---
    print("모델 학습 시작...")
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        train_bar = tqdm(train_loader, desc=f"[Trial {trial_num+1}, Epoch {epoch+1}] Training", leave=False)
        for user_ids, item_ids, ratings in train_bar:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            optimizer.zero_grad()
            predictions = model(user_ids, item_ids)
            loss = criterion(predictions, ratings)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            train_bar.set_postfix(loss=loss.item())

        # --- Validation after each epoch ---
        val_mae, val_mse, val_rmse, val_mape = evaluate_model(model, val_loader, device)
        print(f"Epoch {epoch+1} | Train Loss: {total_train_loss / len(train_loader):.4f} | "
              f"Val RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}")

        # Early stopping logic for this trial
        if val_rmse < trial_best_val_rmse - min_delta:
            trial_best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), trial_model_save_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print("Early stopping triggered for this trial.")
                break
    
    # --- Evaluate the best model from this trial on the Test Set ---
    if os.path.exists(trial_model_save_path):
        model.load_state_dict(torch.load(trial_model_save_path))
        print(f"\nLoaded best model for trial {trial_num+1} from {trial_model_save_path}")
        test_mae, test_mse, test_rmse, test_mape = evaluate_model(model, test_loader, device)
    else:
        print(f"\nWarning: Best model for trial {trial_num+1} not found. Testing with last model state.")
        test_mae, test_mse, test_rmse, test_mape = evaluate_model(model, test_loader, device)

    print(f"\n✅ Trial {trial_num+1} Test Results:")
    print(f" - MSE  : {test_mse:.4f}")
    print(f" - RMSE : {test_rmse:.4f}")
    print(f" - MAE  : {test_mae:.4f}")
    print(f" - MAPE : {test_mape:.2f}%")

    # Store results for this trial
    trial_results = {
        'trial_num': trial_num + 1,
        'parameters': current_params,
        'test_mse': float(test_mse),
        'test_rmse': float(test_rmse),
        'test_mae': float(test_mae),
        'test_mape': float(test_mape)
    }
    results_log.append(trial_results)

    # Check if this trial yielded the overall best RMSE
    if test_rmse < best_rmse:
        best_rmse = test_rmse
        best_params = current_params
        torch.save(model.state_dict(), 'best_overall_neumf_model.pt')
        print(f"  --> New overall best RMSE found: {best_rmse:.4f} with params: {best_params}")

    # Clean up the temporary model file for this trial
    if os.path.exists(trial_model_save_path):
        os.remove(trial_model_save_path)

# -------------------- Final Results Output --------------------
print(f"\n--- Hyperparameter Search Completed ---")
print(f"Overall Best RMSE found: {best_rmse:.4f}")
print(f"Optimal Parameters: {best_params}")

# Save the full results log to a JSON file
with open('neumf_hyperparameter_search_results.json', 'w') as f:
    json.dump(results_log, f, indent=4)
print(f"Full results logged to 'neumf_hyperparameter_search_results.json'")
print(f"Overall best model weights saved to 'best_overall_neumf_model.pt'")

Using device: cuda
✅ Data loaded and preprocessed successfully.

--- Starting Randomized Hyperparameter Search with 10 trials ---
Data Split Random State: 42
Fixed Data Split: Train=313456, Val=44780, Test=89560

==================== Trial 1/10 ====================
Current Parameters: {'mf_dim': 32, 'mlp_dims': [128, 64, 32], 'learning_rate': 0.0005, 'batch_size': 256, 'patience': 5}
모델 학습 시작...


Epoch 1 | Train Loss: 2.0312 | Val RMSE: 1.1586, MAE: 0.9284


Epoch 2 | Train Loss: 1.2605 | Val RMSE: 1.1252, MAE: 0.8877


Epoch 3 | Train Loss: 1.1770 | Val RMSE: 1.1092, MAE: 0.8901


Epoch 4 | Train Loss: 1.1163 | Val RMSE: 1.0971, MAE: 0.8650


...

Epoch 12 | Train Loss: 0.7535 | Val RMSE: 1.1346, MAE: 0.8912


Epoch 13 | Train Loss: 0.7115 | Val RMSE: 1.1461, MAE: 0.9018


Epoch 14 | Train Loss: 0.6709 | Val RMSE: 1.1568, MAE: 0.9086


Epoch 15 | Train Loss: 0.6321 | Val RMSE: 1.1658, MAE: 0.9177


Epoch 16 | Train Loss: 0.5942 | Val RMSE: 1.1798, MAE: 0.9267
Early stopping triggered for this trial.

Loaded best model for trial 10 from temp_neumf_model_trial_10.pt

✅ Trial 10 Test Results:
 - MSE  : 1.1948
 - RMSE : 1.0931
 - MAE  : 0.8574
 - MAPE : 35.03%

--- Hyperparameter Search Completed ---
Overall Best RMSE found: 1.0766
Optimal Parameters: {'mf_dim': 32, 'mlp_dims': [64, 32], 'learning_rate': 0.002, 'batch_size': 128, 'patience': 7}
Full results logged to 'neumf_hyperparameter_search_results.json'
Overall best model weights saved to 'best_overall_neumf_model.pt'


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import sys

# -------------------- 데이터 로딩 및 전처리 --------------------
# 'review_data_optimized.parquet' 파일을 읽어옵니다.
try:
    df = pd.read_parquet('review_data_optimized.parquet', engine='pyarrow')
    df_processed = df[['user_id', 'business_id', 'stars']].copy()

    # Label Encoding을 사용하여 user_id와 business_id를 정수형으로 변환합니다.
    user_encoder = LabelEncoder()
    business_encoder = LabelEncoder()
    df_processed['user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
    df_processed['business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

    num_users = len(user_encoder.classes_)
    num_businesses = len(business_encoder.classes_)
    print("✅ 데이터 로딩 및 전처리 완료.")

except FileNotFoundError:
    print("❌ Error: 'review_data_optimized.parquet' 파일이 존재하지 않습니다.")
    sys.exit()

# -------------------- Dataset 정의 --------------------
class NeuMFDataset(Dataset):
    def __init__(self, df):
        self.user_ids = torch.tensor(df['user_encoded'].values, dtype=torch.long)
        self.item_ids = torch.tensor(df['business_encoded'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['stars'].values, dtype=torch.float)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

# -------------------- 모델 정의 (최적 하이퍼파라미터 적용) --------------------
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, mf_dim=32, mlp_dims=[64, 32]):
        super(NeuMF, self).__init__()
        self.user_embedding_gmf = nn.Embedding(num_users, mf_dim)
        self.item_embedding_gmf = nn.Embedding(num_items, mf_dim)
        
        # mlp_dims[0] 값에 따라 입력 차원 조정
        self.user_embedding_mlp = nn.Embedding(num_users, mlp_dims[0] // 2)
        self.item_embedding_mlp = nn.Embedding(num_items, mlp_dims[0] // 2)

        mlp_layers = []
        input_dim = mlp_dims[0]
        for dim in mlp_dims[1:]:
            mlp_layers.append(nn.Linear(input_dim, dim))
            mlp_layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*mlp_layers)

        self.final_layer = nn.Linear(mf_dim + mlp_dims[-1], 1)

    def forward(self, user_ids, item_ids):
        gmf_user = self.user_embedding_gmf(user_ids)
        gmf_item = self.item_embedding_gmf(item_ids)
        gmf_output = gmf_user * gmf_item

        mlp_user = self.user_embedding_mlp(user_ids)
        mlp_item = self.item_embedding_mlp(item_ids)
        mlp_input = torch.cat((mlp_user, mlp_item), dim=1)
        mlp_output = self.mlp(mlp_input)

        concat = torch.cat((gmf_output, mlp_output), dim=1)
        prediction = self.final_layer(concat)
        return prediction.view(-1)

# -------------------- 평가 지표 함수 정의 --------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# -------------------- 5회 반복 학습 및 평가 (최적 하이퍼파라미터 적용) --------------------
num_runs = 5
mse_list, rmse_list, mae_list, mape_list = [], [], [], []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 최적의 하이퍼파라미터
mf_dim = 32
mlp_dims = [64, 32]
learning_rate = 0.002
batch_size = 128
patience = 7
min_delta = 0.0001
epochs = 50

for i in range(num_runs):
    print(f"\n==================== {i+1}번째 반복 시작 ====================")
    print(f"하이퍼파라미터: MF Dim={mf_dim}, MLP Dims={mlp_dims}, LR={learning_rate}, Batch Size={batch_size}, Patience={patience}")

    # 매 반복마다 데이터셋을 무작위로 분할하여 독립적인 테스트를 보장합니다.
    random_state = 42 + i
    train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=random_state)
    val_size_ratio = 1 / 8
    train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=random_state)

    train_dataset = NeuMFDataset(train_df)
    val_dataset = NeuMFDataset(val_df)
    test_dataset = NeuMFDataset(test_df)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # 모델, 손실 함수, 옵티마이저는 매번 새로 초기화합니다.
    model = NeuMF(num_users, num_businesses, mf_dim=mf_dim, mlp_dims=mlp_dims).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model_path = f'best_neumf_model_run_{i+1}.pt'

    best_val_rmse = float('inf')
    epochs_no_improve = 0

    # 학습 루프
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        train_bar = tqdm(train_loader, desc=f"[Run {i+1}, Epoch {epoch+1}] Training", leave=False)
        for user_ids, item_ids, ratings in train_bar:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            optimizer.zero_grad()
            predictions = model(user_ids, item_ids)
            loss = criterion(predictions, ratings)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            train_bar.set_postfix(loss=loss.item())

        model.eval()
        val_predictions, val_true = [], []
        with torch.no_grad():
            for user_ids, item_ids, ratings in val_loader:
                user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
                preds = model(user_ids, item_ids)
                val_predictions.extend(preds.tolist())
                val_true.extend(ratings.tolist())

        val_rmse = np.sqrt(mean_squared_error(val_true, val_predictions))

        print(f"Epoch {epoch+1} | Train Loss: {total_train_loss / len(train_loader):.4f} | Val RMSE: {val_rmse:.4f}")

        # 조기 종료 (Early Stopping) 로직
        if val_rmse < best_val_rmse - min_delta:
            best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
            print(f"  --> 개선됨. 모델 저장됨 (RMSE: {best_val_rmse:.4f})")
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"조기 종료 발생. (Run {i+1})")
                break

    # 테스트 루프
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f"최적 모델 로드 완료: {model_path}")
    else:
        print(f"최적 모델을 찾지 못해 현재 모델을 사용합니다. (Run {i+1})")

    model.eval()
    test_preds, test_true = [], []
    with torch.no_grad():
        for user_ids, item_ids, ratings in test_loader:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            preds = model(user_ids, item_ids)
            test_preds.extend(preds.tolist())
            test_true.extend(ratings.tolist())

    test_mse = mean_squared_error(test_true, test_preds)
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(test_true, test_preds)
    test_mape = mean_absolute_percentage_error(test_true, test_preds)

    print(f"\n✅ [NeuMF] {i+1}번째 테스트 결과:")
    print(f" - MSE  : {test_mse:.4f}")
    print(f" - RMSE : {test_rmse:.4f}")
    print(f" - MAE  : {test_mae:.4f}")
    print(f" - MAPE : {test_mape:.2f}%")

    mse_list.append(test_mse)
    rmse_list.append(test_rmse)
    mae_list.append(test_mae)
    mape_list.append(test_mape)
    
    # 임시 모델 파일 삭제
    if os.path.exists(model_path):
        os.remove(model_path)
        print(f"임시 모델 파일 삭제: {model_path}")

# 최종 평균 계산 및 출력
avg_mse = np.mean(mse_list)
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print("\n\n==================== 5회 반복 최종 평균 결과 ====================")
print(f" - 평균 MSE  : {avg_mse:.4f}")
print(f" - 평균 RMSE : {avg_rmse:.4f}")
print(f" - 평균 MAE  : {avg_mae:.4f}")
print(f" - 평균 MAPE : {avg_mape:.2f}%")

✅ 데이터 로딩 및 전처리 완료.

==================== 1번째 반복 시작 ====================
하이퍼파라미터: MF Dim=32, MLP Dims=[64, 32], LR=0.002, Batch Size=128, Patience=7


Epoch 1 | Train Loss: 1.6381 | Val RMSE: 1.1074
  --> 개선됨. 모델 저장됨 (RMSE: 1.1074)


Epoch 2 | Train Loss: 1.1352 | Val RMSE: 1.0801
  --> 개선됨. 모델 저장됨 (RMSE: 1.0801)


Epoch 3 | Train Loss: 1.0164 | Val RMSE: 1.0776
  --> 개선됨. 모델 저장됨 (RMSE: 1.0776)


Epoch 4 | Train Loss: 0.9021 | Val RMSE: 1.0947


...

Epoch 6 | Train Loss: 0.6151 | Val RMSE: 1.1652


Epoch 7 | Train Loss: 0.4960 | Val RMSE: 1.1917


Epoch 8 | Train Loss: 0.3995 | Val RMSE: 1.2180


Epoch 9 | Train Loss: 0.3229 | Val RMSE: 1.2455


Epoch 10 | Train Loss: 0.2659 | Val RMSE: 1.2652
조기 종료 발생. (Run 5)
최적 모델 로드 완료: best_neumf_model_run_5.pt

✅ [NeuMF] 5번째 테스트 결과:
 - MSE  : 1.1677
 - RMSE : 1.0806
 - MAE  : 0.8544
 - MAPE : 34.10%
임시 모델 파일 삭제: best_neumf_model_run_5.pt


==================== 5회 반복 최종 평균 결과 ====================
 - 평균 MSE  : 1.1669
 - 평균 RMSE : 1.0802
 - 평균 MAE  : 0.8528
 - 평균 MAPE : 34.05%
